In [1]:
!python -m pip install -U --pre astroquery[all]

     |████████████████████████████████| 5.3 MB 10.2 MB/s eta 0:00:01
     |████████████████████████████████| 287 kB 18.1 MB/s eta 0:00:01
     |████████████████████████████████| 1.9 MB 14.1 MB/s eta 0:00:01
     |████████████████████████████████| 86 kB 10.5 MB/s eta 0:00:01
     |████████████████████████████████| 139 kB 24.7 MB/s eta 0:00:01
     |████████████████████████████████| 1.3 MB 44.9 MB/s eta 0:00:01
     |████████████████████████████████| 12.3 MB 7.4 MB/s eta 0:00:01
     |████████████████████████████████| 82 kB 2.9 MB/s  eta 0:00:01
     |████████████████████████████████| 7.0 MB 32.7 MB/s eta 0:00:01
     |████████████████████████████████| 342 kB 20.2 MB/s eta 0:00:01
  Attempting uninstall: pyerfa
    Found existing installation: pyerfa 1.7.3
    Uninstalling pyerfa-1.7.3:
      Successfully uninstalled pyerfa-1.7.3
  Attempting uninstall: astropy
    Found existing installation: astropy 4.2.1
    Uninstalling astropy-4.2.1:
      Successfully uninstalled astropy-4.2.1
  At

In [2]:
!python -m pip install -U --pre spectres

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from astropy.coordinates import SkyCoord
import astropy.coordinates as coord
import astropy.units as units
from astropy.io import fits
from astropy.io.fits import getdata
from astroquery.sdss import SDSS
import astropy.coordinates as coords
from astropy import units as u
from urllib.error import HTTPError
import os as os
from os.path import exists
from os import system
import matplotlib.patches as patches
from matplotlib.patches import Rectangle
from matplotlib.lines import Line2D
from spectres import spectres
import scipy.integrate as spi

def load_SDSS_spectrum(filename):
    if exists(filename):
        spectrum = getdata(filename)
    elif exists(filename + '.gz'):
        spectrum = getdata(filename + '.gz')
    wl = 10**spectrum['loglam']
    flux = spectrum['flux']
    err = 1./np.sqrt(spectrum['ivar'])
    return wl, flux, err

def get_SDSS_spectrum(plate, mjd, fiberid, load=True):
    # Check if spectrum exists in Data directory:
    data_path = "/Volumes/vtq251/SDSS/PAQSNorth/"
    SDSS_file = "spec-{plate:0=4d}-{mjd}-{fiberid:0=4d}.fits".format(plate=plate, mjd=mjd, fiberid=fiberid)
    if exists(data_path+SDSS_file) or exists(data_path+SDSS_file+'.gz'):
        if load:
            if exists(data_path+SDSS_file):
                wl, flux, err = load_SDSS_spectrum(data_path + SDSS_file)
            else:
                wl, flux, err = load_SDSS_spectrum(data_path + SDSS_file+'.gz')
            return wl, flux, err
        else:
            return
    else:
        # Create file identifier:
        identifier = "{plate:0=4d}/spec-{plate:0=4d}-{mjd}-{fiberid:0=4d}.fits".format(plate=plate,
                                                                                       mjd=mjd,
                                                                                       fiberid=fiberid)
        # Choose server:
        if plate < 3510:
            server = "http://data.sdss3.org/sas/dr12/sdss/spectro/redux/26/spectra/"
            #server = "http://dr16.sdss.org/sas/dr16/sdss/spectro/redux/104/spectra/"
        else:
            # server = "http://data.sdss3.org/sas/dr12/boss/spectro/redux/v5_7_0/spectra/"
            server = "http://dr14.sdss.org/sas/dr14/sdss/spectro/redux/v5_10_0/spectra/"
        # Execute download command:
        try:
            download = "wget -P {output} {server}{ID}".format(output=data_path, server=server, ID=identifier)
            system(download)
        except:
            server = "http://data.sdss3.org/sas/dr12/boss/spectro/redux/v5_7_2/spectra/"
            download = "wget -P {output} {server}{ID}".format(output=data_path, server=server, ID=identifier)
            system(download)
        # -- Compress downloaded spectrum:
        # compression = "gzip {path}/{name}".format(path=data_path, name=SDSS_file)
        # system(compression)
        if load:
            # Now load spectrum:
            wl, flux, err = load_SDSS_spectrum(data_path + SDSS_file)
            return wl, flux, err
        else:
            print('Failed getting:',plate, mjd,fiberid)
            return

#Kilde til data:
# GAIA: https://www.cosmos.esa.int/web/gaia/data-release-3
# SDSS: https://astro.dur.ac.uk/Cosmology/vstatlas/
# UKIDSS: http://casu.ast.cam.ac.uk/vistasp/viking
# WISE: https://wise2.ipac.caltech.edu/docs/release/allwise/

# Den sydlige galaktiske pol: https://astronomy.swin.edu.au/cosmos/s/South+Galactic+Pole

data = np.loadtxt('PAQSsurveyphotometry.dat', skiprows=1)
ID = data[:,0]              #Rektascension fra GAIA kataloget
RA = data[:,1]              #Rektascension fra GAIA kataloget
Dec = data[:,2]             #Deklination fra GAIA kataloget

u_SDSS = data[:,3]          # u-filter størrelsesklasse (SDSS)
err_u_SDSS = data[:,4]        # usikkerhed på u-filter størrelsesklasse (SDSS)
g_SDSS = data[:,5]          # g-filter størrelsesklasse (SDSS)
err_g_SDSS = data[:,6]        # usikkerhed på g-filter størrelsesklasse (SDSS)
r_SDSS = data[:,7]          # r-filter størrelsesklasse (SDSS)
err_r_SDSS = data[:,8]       # usikkerhed på r-filter størrelsesklasse (SDSS)
i_SDSS = data[:,9]         # i-filter størrelsesklasse (SDSS)
err_i_SDSS = data[:,10]       # usikkerhed på i-filter størrelsesklasse (SDSS)
z_SDSS = data[:,11]         # z-filter størrelsesklasse (SDSS)
err_z_SDSS = data[:,12]       # usikkerhed på z-filter størrelsesklasse (SDSS)
u_ATLAS = data[:,13]          # u-filter størrelsesklasse (ATLAS)
err_u_ATLAS = data[:,14]        # usikkerhed på u-filter størrelsesklasse (ATLAS)
g_ATLAS = data[:,15]          # g-filter størrelsesklasse (ATLAS)
err_g_ATLAS = data[:,16]        # usikkerhed på g-filter størrelsesklasse (ATLAS)
r_ATLAS = data[:,17]          # r-filter størrelsesklasse (ATLAS)
err_r_ATLAS = data[:,18]       # usikkerhed på r-filter størrelsesklasse (ATLAS)
i_ATLAS = data[:,19]         # i-filter størrelsesklasse (ATLAS)
err_i_ATLAS = data[:,20]       # usikkerhed på i-filter størrelsesklasse (ATLAS)
z_ATLAS = data[:,21]         # z-filter størrelsesklasse (ATLAS)
err_z_ATLAS = data[:,22]       # usikkerhed på z-filter størrelsesklasse (ATLAS)
y_UKIDSS = data[:,23]
err_y_UKIDSS = data[:,24]
j_UKIDSS = data[:,25]
err_j_UKIDSS = data[:,26]
h_UKIDSS = data[:,27]
err_h_UKIDSS = data[:,28]
ks_UKIDSS = data[:,29]
err_ks_UKIDSS = data[:,30]
y_VHS = data[:,31]
err_y_VHS = data[:,32]
j_VHS = data[:,33]
err_j_VHS = data[:,34]
h_VHS = data[:,35]
err_h_VHS = data[:,36]
ks_VHS = data[:,37]
err_ks_VHS = data[:,38]
W1mag = data[:,39]        # W1-filter størrelsesklasse (WISE)
e_W1mag = data[:,40]      # usikkerhed på W1-filter størrelsesklasse (WISE)
W2mag = data[:,41]        # W2-filter størrelsesklasse (WISE)
e_W2mag = data[:,42]      # usikkerhed på W1-filter størrelsesklasse (WISE)
W3mag = data[:,43]        # W3-filter størrelsesklasse (WISE)
e_W3mag = data[:,44]      # usikkerhed på W1-filter størrelsesklasse (WISE)
W4mag = data[:,45]        # W4-filter størrelsesklasse (WISE)
e_W4mag = data[:,46]      # usikkerhed på W1-filter størrelsesklasse (WISE)

speclist = np.loadtxt('spectraoverview.txt', skiprows=1)
specyn = speclist[:,1]

#Information of filtre og AB-offsets
lambphot = [3540., 4750., 6220., 7630., 9050., 10310., 12480., 16310., 22010., 34000., 46000., 120000., 220000.]
aboffsets = np.asarray([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
#aboffsets = np.asarray([0., 0., 0., 0., 0., 0.634, 0.91, 1.39, 1.85, 2.67, 3.31, 5.14, 6.61])

#Lav farve-farve plottet
errlim1 = 0.20
errlim2 = 0.50
#For NOT run 2024
#good = np.where((r_SDSS < 18.5) & (RA > 200.) & (err_g_SDSS < errlim2) & (err_r_SDSS < errlim1) & (err_j_UKIDSS < errlim2) & (err_ks_UKIDSS < errlim2) & (specyn ==0))
#good2 = np.where((r_SDSS < 18.5) & (RA > 200.) & (err_g_SDSS < errlim2) & (err_r_SDSS < errlim1) & (err_j_VHS < errlim2) & (err_ks_VHS < errlim2) & (specyn ==0))
#goodspec = np.where((r_SDSS < 19.5) & (RA > 220.) & (specyn ==1))
good = np.where((err_g_SDSS < errlim2) & (err_r_SDSS < errlim1) & (err_j_UKIDSS < errlim2) & (err_ks_UKIDSS < errlim2))
good2 = np.where((err_g_SDSS < errlim2) & (err_r_SDSS < errlim1) & (err_j_VHS < errlim2) & (err_ks_VHS < errlim2))
goodspec = np.where((specyn ==1))

def LYMAN_BLANKET(lambd, z):
    #Based on Madau https://articles.adsabs.harvard.edu/pdf/1995ApJ...441...18M
    import numpy as np
    nl = 8
    p = 3.46
    npts = len(lambd)
    y = np.zeros((npts, nl), dtype=float)
    lyman = 1. - 1/(np.arange(nl) + 2)**2
    a = np.array([-0.00360, -0.00170, -0.00120, -0.00093,
                  -0.00090, -0.00087, -0.00085, -0.00080])

    c = 0.
    for i in range(nl):
        c += a[i] * ((1+z) * lyman[i])**p
    c = np.exp(c)

    lyman = 911.753 / lyman

    for i in range(nl):
        y[:, i] = a[i] * (lambd / lyman[i])**p * (lambd <= (1+z) * lyman[i])
    y = np.exp(nl * np.sum(y, axis=1) / npts)

    q = np.where(lambd <= (1+z) * lyman[nl-1])
    if len(q[0]) > 0:
        y[q] = c * np.exp(-0.0015 * z * ((1+z) * lyman[nl-1] - lambd[q]))

    return y


#Read transmission curves for g, r, J, Ks
tab = np.loadtxt('SDSS_g.data')
wave_g = tab[:,0]
transm_g = tab[:,1]

tab = np.loadtxt('SDSS_r.data')
wave_r = tab[:,0]
transm_r = tab[:,1]

tab = np.loadtxt('J_UKIRT.dat')
wave_J = 10.*tab[:,0]
transm_J = tab[:,1]/100.
wave_J = wave_J[::-1]
transm_J = transm_J[::-1]

tab = np.loadtxt('K_UKIRT.dat')
wave_Ks = 10.*tab[:,0]
transm_Ks = tab[:,1]/100.
wave_Ks = wave_Ks[::-1]
transm_Ks = transm_Ks[::-1]

#Filter characteristics from Hewett table 7
lamdacen_g = 4670.
lambdacen_r = 6156.
lambdacen_J = 1248.
lambdacen_Ks = 2201.
ABoff_g = -0.103
ABoff_r = 0.146
ABoff_J = 0.938
ABoff_Ks = 1.900

#SMC extinction parameters
ai = np.array([185.,27.,0.005,0.010,0.012,0.030])
wli = np.array([0.042,0.08,0.22,9.7,18.,25.])
bi = np.array([90.,5.50,-1.95,-1.95,-1.80,0.0])
ni = np.array([2.0,4.0,2.0,2.0,2.0,2.0])
Ki = np.array([2.89,0.91,0.02,1.55,1.72,1.89])

#Read quasar template
tab = np.loadtxt('compoM.data')
wave_qso = tab[:,0]
spec_qso = tab[:,1]

#Read LRD spectra
z = 3.438
#from 1.1 to 1.6
HDU = fits.open('ceers-ddt-v2_prism-clear_2750_1034.spec.fits')
print(HDU.info())
hdr = HDU[1].header
spec1d = HDU[1].data
wave_lrd1 = spec1d.wave*10000./(1+z)
spec_lrd1 = spec1d.flux/(wave_lrd1/10000.)**2
z = 5.05
#from 1.9 to 2.5
HDU = fits.open('jades-gdn-v2_prism-clear_1181_68797.spec.fits')
print(HDU.info())
hdr = HDU[1].header
spec1d = HDU[1].data
wave_lrd2 = spec1d.wave*10000./(1+z)
spec_lrd2 = spec1d.flux/(wave_lrd2/10000.)**2
z=4.53
#from 1.7 to 2.3
HDU = fits.open('macsj0647-v2_prism-clear_1433_1045.spec.fits')
print(HDU.info())
hdr = HDU[1].header
spec1d = HDU[1].data
wave_lrd3 = spec1d.wave*10000./(1+z)
spec_lrd3 = spec1d.flux/(wave_lrd3/10000.)**2

#Read galaxy templates
tab = np.loadtxt('lrt_templates.dat')
wave_jaffet = tab[:,0]*10000.
spec_E = tab[:,3]/wave_jaffet**2
spec_Sbc = tab[:,4]/wave_jaffet**2
spec_I = tab[:,5]/wave_jaffet**2

#Rebin to quasar wavelength grid
spec_I_rebin = spectres(wave_qso,wave_jaffet,spec_I)
spec_Sbc_rebin = spectres(wave_qso,wave_jaffet,spec_Sbc)
spec_E_rebin = spectres(wave_qso,wave_jaffet,spec_E)
spec_gal_rebin = spec_I_rebin

#LRD1:
#Spec_I
#1/9 weight to gal/qso
#Avqso = 0.0
#Avgal = 0.
#normalise at 5250-5600 Å
#select LRD
AVqso = 0.0
AVgal = 0.0
wave_lrd = wave_lrd1
spec_lrd = spec_lrd1
#Redden the qso template
ABqso = AVqso*4.1/3.1
Alambda = spec_qso*0.
wlr = wave_qso/1.e4
for e in range(len(ai)):
    Alambda=Alambda+ai[e]/((wlr/wli[e])**ni[e]+(wli[e]/wlr)**ni[e]+bi[e])
Alambda = Alambda*ABqso
model = 10**(-0.4*Alambda)*spec_qso
#Redden the gal template
ABgal = AVgal*4.1/3.1
Alambda = spec_qso*0.
wlr = wave_qso/1.e4
for e in range(len(ai)):
    Alambda=Alambda+ai[e]/((wlr/wli[e])**ni[e]+(wli[e]/wlr)**ni[e]+bi[e])
Alambda = Alambda*ABgal
spec_gal_rebin = 10**(-0.4*Alambda)*spec_gal_rebin
#Normalise
specfilt = np.nonzero((wave_lrd > 5250) & (wave_lrd < 5600))
normspec = np.mean(spec_lrd[specfilt])
modelfilt = np.nonzero((wave_qso > 5250) & (wave_qso < 5600))
normqso = np.mean(model[modelfilt])
normgal = np.mean(spec_gal_rebin[modelfilt])
factorqso = normspec/normqso
factorgal = normspec/normgal
model_lrd1 = (9.*model*factorqso+1.*spec_gal_rebin*factorgal)/10.

#LRD2:
#Spec_I
#1/3 weight to gal/qso
#Avqso = 4.5
#Avgal = 0.
#normalise at 5250-5600 Å
#select LRD
AVqso = 4.5
AVgal = 0.0
wave_lrd = wave_lrd2
spec_lrd = spec_lrd2
#Redden the qso template
ABqso = AVqso*4.1/3.1
Alambda = spec_qso*0.
wlr = wave_qso/1.e4
for e in range(len(ai)):
    Alambda=Alambda+ai[e]/((wlr/wli[e])**ni[e]+(wli[e]/wlr)**ni[e]+bi[e])
Alambda = Alambda*ABqso
model = 10**(-0.4*Alambda)*spec_qso
#Redden the gal template
ABgal = AVgal*4.1/3.1
Alambda = spec_qso*0.
wlr = wave_qso/1.e4
for e in range(len(ai)):
    Alambda=Alambda+ai[e]/((wlr/wli[e])**ni[e]+(wli[e]/wlr)**ni[e]+bi[e])
Alambda = Alambda*ABgal
spec_gal_rebin = 10**(-0.4*Alambda)*spec_gal_rebin
#Normalise
specfilt = np.nonzero((wave_lrd > 5250) & (wave_lrd < 5600))
normspec = np.mean(spec_lrd[specfilt])
modelfilt = np.nonzero((wave_qso > 5250) & (wave_qso < 5600))
normqso = np.mean(model[modelfilt])
normgal = np.mean(spec_gal_rebin[modelfilt])
factorqso = normspec/normqso
factorgal = normspec/normgal
model_lrd2 = (3.*model*factorqso+1.*spec_gal_rebin*factorgal)/4.

#LRD3:
#Spec_I
#4/1 weight to gal/qso
#Avqso = 8.0
#Avgal = -0.2
#normalise at 5250-5600 Å
#select LRD
AVqso = 8.0
AVgal = -0.2
wave_lrd = wave_lrd3
spec_lrd = spec_lrd3
#Redden the qso template
ABqso = AVqso*4.1/3.1
Alambda = spec_qso*0.
wlr = wave_qso/1.e4
for e in range(len(ai)):
    Alambda=Alambda+ai[e]/((wlr/wli[e])**ni[e]+(wli[e]/wlr)**ni[e]+bi[e])
Alambda = Alambda*ABqso
model = 10**(-0.4*Alambda)*spec_qso
#Redden the gal template
ABgal = AVgal*4.1/3.1
Alambda = spec_qso*0.
wlr = wave_qso/1.e4
for e in range(len(ai)):
    Alambda=Alambda+ai[e]/((wlr/wli[e])**ni[e]+(wli[e]/wlr)**ni[e]+bi[e])
Alambda = Alambda*ABgal
spec_gal_rebin = 10**(-0.4*Alambda)*spec_gal_rebin
#Normalise
specfilt = np.nonzero((wave_lrd > 5250) & (wave_lrd < 5600))
normspec = np.mean(spec_lrd[specfilt])
modelfilt = np.nonzero((wave_qso > 5250) & (wave_qso < 5600))
normqso = np.mean(model[modelfilt])
normgal = np.mean(spec_gal_rebin[modelfilt])
factorqso = normspec/normqso
factorgal = normspec/normgal
model_lrd3 = (1.*model*factorqso+4.*spec_gal_rebin*factorgal)/5.


#Read spectrum of vega
tab = np.loadtxt('vega.dat')
wave_vega = tab[:,0]
spec_vega = tab[:,1]*1.e17

#Integrate the fluxes of vega in the four filters
spec_vega_resample_g =  spectres(wave_g,wave_vega,spec_vega*wave_vega)
gvega = spi.simps(spec_vega_resample_g*transm_g, wave_g)/spi.simps(transm_g*wave_g, wave_g)
spec_vega_resample_r =  spectres(wave_r,wave_vega,spec_vega*wave_vega)
rvega = spi.simps(spec_vega_resample_r*transm_r, wave_r)/spi.simps(transm_r*wave_r, wave_r)
spec_vega_resample_J =  spectres(wave_J,wave_vega,spec_vega*wave_vega)
Jvega = spi.simps(spec_vega_resample_J*transm_J, wave_J)/spi.simps(transm_J*wave_J, wave_J)
spec_vega_resample_Ks =  spectres(wave_Ks,wave_vega,spec_vega*wave_vega)
Ksvega = spi.simps(spec_vega_resample_Ks*transm_Ks, wave_Ks)/spi.simps(transm_Ks*wave_Ks, wave_Ks)

gr_modellrd3 = np.empty(60, dtype=float)
JKs_modellrd3 = np.empty(60, dtype=float)
for nz in range(0,60):
    z = float(nz)/10.+0.0
    wave_qso_z = wave_qso*(1.+z)
    spec_qso = model_lrd3*LYMAN_BLANKET(wave_qso_z,z)
    spec_qso_resample_g =  spectres(wave_g,wave_qso_z,spec_qso*wave_qso_z)
    spec_qso_resample_r =  spectres(wave_r,wave_qso_z,spec_qso*wave_qso_z)
    spec_qso_resample_J =  spectres(wave_J,wave_qso_z,spec_qso*wave_qso_z)
    spec_qso_resample_Ks =  spectres(wave_Ks,wave_qso_z,spec_qso*wave_qso_z)
    gflam = spi.simps(spec_qso_resample_g*transm_g, wave_g)/spi.simps(transm_g*wave_g, wave_g)
    rflam = spi.simps(spec_qso_resample_r*transm_r, wave_r)/spi.simps(transm_r*wave_r, wave_r)
    Jflam = spi.simps(spec_qso_resample_J*transm_J, wave_J)/spi.simps(transm_J*wave_J, wave_J)
    Ksflam = spi.simps(spec_qso_resample_Ks*transm_Ks, wave_Ks)/spi.simps(transm_Ks*wave_Ks, wave_Ks)
    gr_modellrd3[nz] = -2.5*np.log10(gflam/rflam*rvega/gvega)+ABoff_g-ABoff_r
    JKs_modellrd3[nz] = -2.5*np.log10(Jflam/Ksflam*Ksvega/Jvega)
    
JKs_lrd3 = np.empty(65, dtype=float)
gr_lrd3=np.empty(65, dtype=float)
for nz in range(0,65):
    z = float(nz)/100.+1.7
    wave_lrd_z = wave_lrd3*(1.+z)
    spec_lrd = spec_lrd3*LYMAN_BLANKET(wave_lrd_z,z)
    spec_lrd_resample_g =  spectres(wave_g,wave_lrd_z,spec_lrd*wave_lrd_z)
    spec_lrd_resample_r =  spectres(wave_r,wave_lrd_z,spec_lrd*wave_lrd_z)
    spec_lrd_resample_J =  spectres(wave_J,wave_lrd_z,spec_lrd*wave_lrd_z)
    spec_lrd_resample_Ks =  spectres(wave_Ks,wave_lrd_z,spec_lrd*wave_lrd_z)
    gflam = spi.simps(spec_lrd_resample_g*transm_g, wave_g)/spi.simps(transm_g*wave_g, wave_g)
    rflam = spi.simps(spec_lrd_resample_r*transm_r, wave_r)/spi.simps(transm_r*wave_r, wave_r)
    Jflam = spi.simps(spec_lrd_resample_J*transm_J, wave_J)/spi.simps(transm_J*wave_J, wave_J)
    Ksflam = spi.simps(spec_lrd_resample_Ks*transm_Ks, wave_Ks)/spi.simps(transm_Ks*wave_Ks, wave_Ks)
    gr_lrd3[nz] = -2.5*np.log10(gflam/rflam*rvega/gvega)+ABoff_g-ABoff_r
    JKs_lrd3[nz] = -2.5*np.log10(Jflam/Ksflam*Ksvega/Jvega)
    
fig,ax = plt.subplots(1,figsize=(10,5),dpi=300)
ax.scatter(j_UKIDSS[good]-ks_UKIDSS[good],g_SDSS[good]-r_SDSS[good],color='black',linewidths=0.3, s=1, label='SDSS/UKIDSS')
ax.scatter(j_VHS[good2]-ks_VHS[good2],g_SDSS[good2]-r_SDSS[good2],color='green',linewidths=0.3, s=1, label='SDSS/VHS')
#ax.scatter(j_UKIDSS[goodspec]-ks_UKIDSS[goodspec],g_SDSS[goodspec]-r_SDSS[goodspec],color='black',linewidths=0.6, s=2, label='with SDSS spectra')
plt.title('Southern sample, g-r vs. J-Ks')
plt.xlabel('J-Ks (UKIDSS/VHS)')
plt.ylabel('g-r (SDSS/ALTLAS)')
plt.xlim(-1.,4)
plt.ylim(-1.,4)
square=Rectangle((1.9,-0.1),1.4,1.7, fill=False,color='cyan',linewidth=2,linestyle='--')
plt.gca().add_patch(square)
#plt.plot(JKs_lrd1, gr_lrd1, 's',color='b', label='LRD1')
#plt.plot(JKs_lrd2, gr_lrd2, 's',color='g', label='LRD2')
plt.plot(JKs_lrd3, gr_lrd3, 's',color='r', label='LRD3')
#plt.plot(JKs_modellrd1, gr_modellrd1, '.',color='lightskyblue', label='model LRD1')
#plt.plot(JKs_modellrd2, gr_modellrd2, '.',color='lightgreen', label='model LRD2')
plt.plot(JKs_modellrd3, gr_modellrd3, '.',color='tomato', label='model LRD3')
ax.legend()
plt.savefig('sdss_lrd3.png') 
plt.show()